In [1]:
import sys
from pathlib import Path

# Add the project root directory to Python path
project_root = Path().absolute().parent
sys.path.append(str(project_root))

# Verify the environment is working
print(f"Python executable: {sys.executable}")
print(f"Project root: {project_root}")
print(f"Python path includes project root: {str(project_root) in sys.path}")

Python executable: /Users/silvanragettli/Python_Environments/snowcover-mapper/venv/bin/python
Project root: /Users/silvanragettli/hydrosolutions Dropbox/Silvan Ragettli/2025_04_GlacierMapper/snowcover-mapper
Python path includes project root: True


In [2]:
import ee
# # Authenticate and initialize Earth Engine
# ee.Authenticate(auth_mode='notebook', force=True)
# ee.Initialize(project="ee-sahellakes")
ee.Initialize(project="thurgau-irrigation")

In [3]:
# Snow Cover Mapping Notebook

# This cell initializes Earth Engine and sets up paths and config

import geemap
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import your project-specific modules from `src`
from src.modis_processing import (load_modis, fill_modis_with_aqua, add_date_bands,create_decadal_composites, extract_year_ranges,process_interval,process_interval_250,
    load_modis_250,fill_modis_with_aqua_250, create_decadal_composites_250,modis_cloud_masking_250
)
from src.dem_processing import load_dem, classify_aspect, reproject_and_analyze_dem
from src.snowline import get_snowline_elevation, calculate_glacier_metrics

# ---------------------------------------------
# Load AOI from RiverBasins_2023 asset
# ---------------------------------------------
RiverBasins_2023 = ee.FeatureCollection('users/hydrosolutions/RiverBasins_CA_Jan2023_simple1000')
RiverBasins_2023 = RiverBasins_2023.map(lambda ft: ft.set('NAME', ee.String(ft.get('BASIN')).cat(ee.String('_')).cat(ee.String(ft.get('CODE')))))
glims = ee.FeatureCollection("GLIMS/20230607").filter(ee.Filter.eq('geog_area', "Randolph Glacier Inventory; Umbrella RC for merging the RGI into GLIMS"));


In [7]:
intervals = ee.List(extract_year_ranges(ee.List.sequence(2022, 2022), 10)
    .iterate(lambda list, previous: ee.List(previous).cat(ee.List(list)), ee.List([])))
print('intervals:',intervals.length().getInfo())
print('intervals:',intervals.getInfo())

intervals: 36
intervals: [[{'type': 'Date', 'value': 1640995200000}, {'type': 'Date', 'value': 1641888000000}], [{'type': 'Date', 'value': 1641888000000}, {'type': 'Date', 'value': 1642780800000}], [{'type': 'Date', 'value': 1642780800000}, {'type': 'Date', 'value': 1643673600000}], [{'type': 'Date', 'value': 1643673600000}, {'type': 'Date', 'value': 1644480000000}], [{'type': 'Date', 'value': 1644480000000}, {'type': 'Date', 'value': 1645286400000}], [{'type': 'Date', 'value': 1645286400000}, {'type': 'Date', 'value': 1646092800000}], [{'type': 'Date', 'value': 1646092800000}, {'type': 'Date', 'value': 1646985600000}], [{'type': 'Date', 'value': 1646985600000}, {'type': 'Date', 'value': 1647878400000}], [{'type': 'Date', 'value': 1647878400000}, {'type': 'Date', 'value': 1648771200000}], [{'type': 'Date', 'value': 1648771200000}, {'type': 'Date', 'value': 1649635200000}], [{'type': 'Date', 'value': 1649635200000}, {'type': 'Date', 'value': 1650499200000}], [{'type': 'Date', 'value': 1

In [4]:

# filter by code 16230
aoi = RiverBasins_2023.filter(ee.Filter.eq('CODE', '15002')).geometry()
print('AOI:', aoi.getInfo())

start_year = 2024
end_year = 2024

# ---------------------------------------------
# Load and process MODIS
# ---------------------------------------------
intervals = ee.List(extract_year_ranges(ee.List.sequence(start_year, end_year), 10)
    .iterate(lambda list, previous: ee.List(previous).cat(ee.List(list)), ee.List([])))

# print('intervals:',intervals)
print('interval:', ee.List(intervals.slice(19,20).get(0)).getInfo())

# Load all MODIS data for the entire time span
start_date = ee.Date.fromYMD(start_year, 1, 1)
end_date = ee.Date.fromYMD(end_year, 12, 31)
terra_coll = load_modis(aoi)

# Create filled MODIS snow cover fraction collection
mscf = terra_coll.map(lambda img: fill_modis_with_aqua(img))
# print('mscf:', mscf.first().getInfo())

# time_intervals_all should be an ee.List of [start, end] ee.Date pairs
modis_ic = ee.ImageCollection(intervals.slice(19,20).map(lambda list:process_interval(mscf, list)))
# print('modis_ic:', modis_ic.filterBounds(aoi).getInfo())

# ---------------------------------------------
# Load MODIS 250 m data (only for testing)
# ---------------------------------------------
terra_coll = load_modis_250(aoi)
# mscf = terra_coll.map(lambda img: modis_cloud_masking_250(img,aoi))
# print('mscf 250:', mscf.first().getInfo())
# # time_intervals_all should be an ee.List of [start, end] ee.Date pairs
# modis_ic_250 = ee.ImageCollection(intervals.slice(19,20).map(lambda list:process_interval_250(mscf, list)))
# print('modis_ic 250:', modis_ic_250.filterBounds(aoi).getInfo())

mscf = terra_coll.map(lambda img: fill_modis_with_aqua_250(img,aoi))
print('mscf 250:', mscf.first().getInfo())
# time_intervals_all should be an ee.List of [start, end] ee.Date pairs
modis_ic_250 = ee.ImageCollection(intervals.slice(19,20).map(lambda list:process_interval_250(mscf, list)))
print('modis_ic 250:', modis_ic_250.filterBounds(aoi).getInfo())

# modis_ic = create_decadal_composites(aoi, start_year, end_year, agg_interval=10)
# print('modis_ic:', modis_ic.filterBounds(aoi).first().getInfo())
# terra, aqua = load_modis(aoi, start_date, end_date)
# filled_modis = terra.map(lambda img: fill_modis_with_aqua(img, aqua))
# filled_modis = filled_modis.map(add_date_bands)

# ---------------------------------------------
# Load DEM and classify terrain aspect
# ---------------------------------------------
dem = load_dem()
modisProjection = modis_ic.filterBounds(aoi).first().projection()
scale = 500
tile_scale = 2
sc_th = 50
aspect_keys = ['East', 'North', 'South', 'West', 'mixed']
# Reproject and analyze DEM
reprojected_dem, min_dem_dict, max_dem_dict, n_grid = reproject_and_analyze_dem(dem, modisProjection, aoi, scale, tile_scale, aspect_keys)
# print('reprojected_dem:', reprojected_dem.getInfo())

aspects, aspect_coded = classify_aspect(dem, modisProjection, scale)
# print('aspects:', aspects.bandNames().getInfo())
# print('n_grid:', n_grid.getInfo())

# ---------------------------------------------
# Snowline elevation estimation from first MODIS image
# ---------------------------------------------
sample_img = ee.Image(modis_ic.filterDate(start_date, end_date).first())

# Test on a problematic summer image
sample_img = modis_ic.filterDate('2024-07-01', '2024-08-01').first()

print('sample_img:', sample_img.getInfo())
# Get snowline elevation and statistics
snowline_stats,fsc = get_snowline_elevation(sample_img, reprojected_dem, aspect_coded, aoi,min_dem_dict, max_dem_dict,n_grid, 
                                                      scale=500, scale_dem=500, sc_th=50, canny_threshold=0.7, 
                           canny_sigma=0.7, ppha=10, tile_scale=1, point2sample=1000, aspectKeys=['East', 'North', 'South', 'West', 'mixed'])
  
print('Snowline elevation by aspect:', snowline_stats.getInfo())
print('Fractional Snow Cover', fsc.getInfo())

glacier_metrics=calculate_glacier_metrics(glims, aoi, sample_img,sc_th, snowline_stats , dem, aspect_keys, tile_scale, aspects)
# Access individual EE values from the dictionary and get their info
# print('Glacier SCF:', glacier_metrics['glims_fsc'].getInfo())
# print('Glacier SCF below snowline:', glacier_metrics['glims_fsc_below_sl'].getInfo())
# print('Glacier area below snowline:', glacier_metrics['glims_area_below_sl'].getInfo())


AOI: {'type': 'Polygon', 'coordinates': [[[78.35558680529317, 42.841624935346175], [78.34886246924157, 42.82958091911706], [78.38582393379298, 42.79144665039145], [78.38308162476883, 42.74938845973651], [78.36565543091028, 42.739132472737566], [78.36738999652627, 42.73449054294478], [78.40500703170778, 42.72508188230116], [78.40886416962566, 42.710366813467374], [78.42423912934738, 42.69919676146307], [78.42649543944474, 42.69001100182323], [78.45619304986197, 42.68484290924801], [78.45496678270888, 42.676847760794644], [78.53783915075857, 42.69240559491001], [78.62981709232788, 42.703526603101125], [78.64310515220191, 42.687103715906815], [78.6819796237287, 42.678399517588296], [78.7454326670329, 42.686457096467194], [78.79148623308848, 42.69238772002842], [78.87993245601564, 42.69640981460532], [78.87640979374024, 42.7201055241284], [78.92885325597925, 42.72002523156944], [78.94693050268994, 42.695950536158875], [78.95391786273467, 42.686341204648464], [78.99936502354535, 42.67334734

### Processing MODIS 500 NSDI DATA PER BASIN

In [ ]:
import datetime
##todo: automatically identify the last export date and continue from there

# Loop over each basin and export results for a given month (or several months)
year=2025
catchment_names=RiverBasins_2023.aggregate_array('NAME').getInfo()
#get the current month
current_month = datetime.datetime.now().month  # returns 1-12

for catchment_name in catchment_names:

    print(f"Processing catchment {catchment_name}...")
    
    # filter by code 16230
    aoi = RiverBasins_2023.filter(ee.Filter.eq('NAME', catchment_name)).geometry()

    # Load MODIS data for the year
    modis_ic = create_decadal_composites(aoi, year, year, agg_interval=10)

    # Define start and end dates for the year: run from the beginning to the previous month from now
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year, current_month - 1, 31)
    
    # Filter by date
    modis_ic = ee.ImageCollection(modis_ic).filter(ee.Filter.And(ee.Filter.calendarRange(year, year, 'year'), ee.Filter.calendarRange(1, current_month - 1, 'month')))
    modisProjection = modis_ic.filterBounds(aoi).first().projection()
    reprojected_dem, min_dem_dict, max_dem_dict, n_grid = reproject_and_analyze_dem(dem, modisProjection, aoi, scale, tile_scale, aspect_keys)
    aspects, aspect_coded = classify_aspect(dem, modisProjection, scale)    
    
    # Function to process each image and create a feature with properties
    def create_feature_with_properties(img):
        # Get snowline elevation for this image
        current_snowline_stats, current_fsc = get_snowline_elevation(
            img, reprojected_dem, aspect_coded, aoi, min_dem_dict, max_dem_dict, n_grid,
            scale=500, scale_dem=500, sc_th=50, canny_threshold=0.7,
            canny_sigma=0.7, ppha=10, tile_scale=1, point2sample=1000, 
            aspectKeys=aspect_keys
        )
        
        # Calculate glacier metrics
        current_glacier_metrics = calculate_glacier_metrics(
            glims, aoi, img, sc_th, current_snowline_stats, dem, aspect_keys, tile_scale, aspects
        )
        
        # Create a dictionary with aspect-specific values
        aspect_dict = {}
        base = ee.String('SLA_')
        
        # Add properties for each aspect (excluding 'mixed')
        for i, aspect in enumerate(aspect_keys[:-1]):
            aspect_dict[base.cat(aspect)] = current_snowline_stats.get(aspect)
        
        # Get date info
        img_date = ee.Date(img.get('system:time_start'))
        img_year = img_date.get('year')
        img_decade = ee.Number(img_date.get('day')).add(2).divide(10).ceil()
        
        # Create feature with all properties
        feature = ee.Feature(None).set(
            'Year-Month-Day', img_date.format('YYYY-MM-dd'),
            'year', img_year,
            'decade', img_decade,
            'gla_fsc', current_glacier_metrics['glims_fsc'],
            'gla_fsc_below_sl50', current_glacier_metrics['glims_fsc_below_sl'],
            'gla_area_below_sl50', current_glacier_metrics['glims_area_below_sl'],
            'fsc', current_fsc
        )
        
        # Add aspect-specific properties
        for key, value in aspect_dict.items():
            feature = feature.set(key, value)
            
        return feature
    
    # Apply the function to each image in the collection
    aoi_mean_tmp = modis_ic.map(create_feature_with_properties)
    
    # Add geometry to features (because null geometry can't be exported)
    joined = aoi_mean_tmp.map(lambda ft: ee.Feature(aoi.centroid(1000)).copyProperties(ft))
    
    # Sort by glacier snow cover below snowline
    table_to_export = joined#.sort('gla_fsc_below_sl50', False)
    
    export_layer_name = 'decadal_SLA'  # Modify as needed
    
    # Create year_month string for asset naming
    year_month = f"{year}_{current_month-1:02d}"
    
    # Export to asset
    task = ee.batch.Export.table.toAsset(
        collection=ee.FeatureCollection(table_to_export).set('NAME', catchment_name),
        description=f"{export_layer_name}_{catchment_name.replace('.', '')}_{year_month}",
        assetId=f"projects/ee-hydro4u/assets/snow_CentralAsia/Folder4SLA_v4/{export_layer_name}_{catchment_name.replace('.', '')}_{year_month}"
    )
    
    # # Start the export task
    task.start()
    print(f"Export started for year {year} and month {current_month-1}.")

Processing catchment ISSYKUL_15054...
Export started for year 2025 and month 8.
Processing catchment ISSYKUL_15002...
Export started for year 2025 and month 8.
Processing catchment ISSYKUL_15013...
Export started for year 2025 and month 8.
Processing catchment ISSYKUL_15016...
Export started for year 2025 and month 8.
Processing catchment ISSYKUL_15020...
Export started for year 2025 and month 8.
Processing catchment ISSYKUL_15022...
Export started for year 2025 and month 8.
Processing catchment ISSYKUL_15025...
Export started for year 2025 and month 8.
Processing catchment ISSYKUL_15030...
Export started for year 2025 and month 8.
Processing catchment ISSYKUL_15034...
Export started for year 2025 and month 8.
Processing catchment ISSYKUL_15039...
Export started for year 2025 and month 8.
Processing catchment ISSYKUL_15040...
Export started for year 2025 and month 8.
Processing catchment ISSYKUL_15044...
Export started for year 2025 and month 8.
Processing catchment ISSYKUL_15045...
Ex

In [ ]:
# Loop through years 2001 to 2024 (annual assets)
for year in range(start_year, end_year + 1):

    print(f"Processing year {year}...")
    
    # Define start and end dates for the year
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = ee.Date.fromYMD(year, 12, 31)
    
    # Load MODIS data for the year
    modis_ic = create_decadal_composites(aoi, year, year, agg_interval=10)
    
    # Load and process DEM if needed (or reuse from outside the loop if not changing)
    dem = load_dem()
    modisProjection = modis_ic.filterBounds(aoi).first().projection()
    
    # Function to process each image and create a feature with properties
    def create_feature_with_properties(img):
        # Get snowline elevation for this image
        current_snowline_stats, current_fsc = get_snowline_elevation(
            img, reprojected_dem, aspect_coded, aoi, min_dem_dict, max_dem_dict, n_grid,
            scale=500, scale_dem=500, sc_th=50, canny_threshold=0.7,
            canny_sigma=0.7, ppha=10, tile_scale=1, point2sample=1000, 
            aspectKeys=aspect_keys
        )
        
        # Calculate glacier metrics
        current_glacier_metrics = calculate_glacier_metrics(
            glims, aoi, img, sc_th, current_snowline_stats, dem, aspect_keys, tile_scale, aspects
        )
        
        # Create a dictionary with aspect-specific values
        aspect_dict = {}
        base = ee.String('SLA_')
        
        # Add properties for each aspect (excluding 'mixed')
        for i, aspect in enumerate(aspect_keys[:-1]):
            aspect_dict[base.cat(aspect)] = current_snowline_stats.get(aspect)
        
        # Get date info
        img_date = ee.Date(img.get('system:time_start'))
        img_year = img_date.get('year')
        img_decade = ee.Number(img_date.get('day')).add(2).divide(10).ceil()
        
        # Create feature with all properties
        feature = ee.Feature(None).set(
            'Year-Month-Day', img_date.format('YYYY-MM-dd'),
            'year', img_year,
            'decade', img_decade,
            'gla_fsc', current_glacier_metrics['glims_fsc'],
            'gla_fsc_below_sl50', current_glacier_metrics['glims_fsc_below_sl'],
            'gla_area_below_sl50', current_glacier_metrics['glims_area_below_sl'],
            'fsc', current_fsc
        )
        
        # Add aspect-specific properties
        for key, value in aspect_dict.items():
            feature = feature.set(key, value)
            
        return feature
    
    # Apply the function to each image in the collection
    aoi_mean_tmp = modis_ic.map(create_feature_with_properties)
    
    # Add geometry to features (because null geometry can't be exported)
    joined = aoi_mean_tmp.map(lambda ft: ee.Feature(aoi.centroid(1000)).copyProperties(ft))
    
    # Sort by glacier snow cover below snowline
    table_to_export = joined#.sort('gla_fsc_below_sl50', False)
    
    # Set catchment name (assuming it's defined elsewhere)
    catchment_name = 'AKHANGARAN_16230'  # Modify as needed
    export_layer_name = 'decadal_SLA'  # Modify as needed
    
    # Export to asset
    task = ee.batch.Export.table.toAsset(
        collection=ee.FeatureCollection(table_to_export).set('NAME', catchment_name),
        description=f"{export_layer_name}_{catchment_name.replace('.', '')}_Year{year}",
        assetId=f"projects/ee-hydro4u/assets/snow_CentralAsia/Folder4SLA_v4/{export_layer_name}_{catchment_name.replace('.', '')}_Year{year}"
    )
    
    # Start the export task
    task.start()
    print(f"Export started for year {year}")

#### MODIS 250m REFLECTANCE TIME SERIES OF MEAN VALUES OVER GLACIERIZED AREA, PER BASIN

In [ ]:
# Check position of catchment in the list
catchment_names=RiverBasins_2023.aggregate_array('NAME').getInfo()
print('catchment_names:', catchment_names)
#position of the catchment with id 16205 in NAME
position=catchment_names.index('SYR_DARYA_16938')
print('position:', position)
position=catchment_names.index('SYR_DARYA_60024')
print('position:', position)

catchment_names: ['ISSYKUL_15054', 'ISSYKUL_15002', 'ISSYKUL_15013', 'ISSYKUL_15016', 'ISSYKUL_15020', 'ISSYKUL_15022', 'ISSYKUL_15025', 'ISSYKUL_15030', 'ISSYKUL_15034', 'ISSYKUL_15039', 'ISSYKUL_15040', 'ISSYKUL_15044', 'ISSYKUL_15045', 'ISSYKUL_15049', 'ISSYKUL_15051', 'ISSYKUL_15057', 'ISSYKUL_15064', 'ISSYKUL_15069', 'ISSYKUL_15070', 'ISSYKUL_15081', 'ISSYKUL_15083', 'ISSYKUL_15090', 'ZERAFSHAN_17461', 'PYANDZH_17043', 'PYANDZH_17045', 'PYANDZH_17047', 'PYANDZH_17050', 'PYANDZH_17058', 'PYANDZH_17059', 'PYANDZH_17062', 'PYANDZH_17073', 'PYANDZH_17078', 'VAKSH_17082', 'VAKSH_17100', 'VAKSH_17101', 'VAKSH_17110', 'KOFARNIKHAN_17137', 'KOFARNIKHAN_17150', 'KOFARNIKHAN_17185', 'KOFARNIKHAN_17202', 'ZERAFSHAN_17288', 'ZERAFSHAN_17325', 'ZERAFSHAN_17329', 'ZERAFSHAN_17338', 'ZERAFSHAN_17344', 'PYANDZH_17391', 'PYANDZH_17453', 'VAKSH_17459', 'VAKSH_17462', 'ZERAFSHAN_16223', 'SURKHANDARYA_17194', 'SURKHANDARYA_17211', 'SURKHANDARYA_17223', 'QASHQADARYA_17231', 'QASHQADARYA_17236', 'QASHQ

In [ ]:
import datetime
##todo: automatically identify the last export date and continue from there

# Loop over each basin and export results for a given month (or several months)
year_first=2001
year_last=2024
catchment_names=RiverBasins_2023.aggregate_array('NAME').getInfo()
print('catchment_names:', catchment_names)
# #get the current month
# current_month = datetime.datetime.now().month  # returns 1-12

# Glacier mask: composite of all available glacier outlines saved as asset by tile (exported using the script create_glacier_mask_tiles.py)
glacier_mask = ee.ImageCollection('projects/ee-hydro4u/assets/snow_CentralAsia/glacier_mask_collection').max()


for catchment_name in catchment_names:
    # filter by code 16230
    aoi = RiverBasins_2023.filter(ee.Filter.eq('NAME', catchment_name)).geometry()

    print(f"Processing catchment {catchment_name}...")

    for year in range(year_first, year_last+1):
        print(f"  Year: {year}")

        # Load MODIS data for the year
        modis_ic = create_decadal_composites_250(aoi, year, year, agg_interval=10)
        # modis_ic = create_decadal_composites_250(aoi, 2025, 2025, agg_interval=10)

        # Filter by date
        modis_ic = ee.ImageCollection(modis_ic).filter(ee.Filter.calendarRange(year, year, 'year'))
        # modis_ic = ee.ImageCollection(modis_ic).filter(ee.Filter.And(ee.Filter.calendarRange(2025, 2025, 'year'), ee.Filter.calendarRange(9,9, 'month')))
           
        # Function to process each image and create a feature with properties
        def create_feature_with_properties(img):

            # Calculate mean NIR value
            mean_NIR_value = img.select('value').updateMask(glacier_mask).reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=aoi,
                scale=250,
                maxPixels=1e13,
                tileScale=1
            ).values()

            # mark by -9999 if mean_NIR_value is null
            mean_NIR_value = ee.Number(ee.Algorithms.If(mean_NIR_value.size().eq(0), -9999, mean_NIR_value.get(0)))

            # Get date info
            img_date = ee.Date(img.get('system:time_start'))
            # img_year = img_date.get('year')
            img_decade = ee.Number(img_date.get('day')).add(2).divide(10).ceil()
            
            # Create feature with all properties
            feature = ee.Feature(None).set(
                'Year-Month-Day', img_date.format('YYYY-MM-dd'),
                'year', year,
                'decade', img_decade,
                'mean_NIR', mean_NIR_value,
                'cc_fraction', img.get('cc_fraction'),
                'cc_fraction2', img.get('cc_fraction2')
            )

            return feature
        
        # Apply the function to each image in the collection
        aoi_mean_tmp = modis_ic.map(create_feature_with_properties)
        
        # print('aoi_mean_tmp:', aoi_mean_tmp.first().getInfo())
        # aoi_mean_tmp: {'type': 'Feature', 'geometry': None, 'id': '15', 'properties': {'Year-Month-Day': '2024-06-01', 'decade': 1, 'mean_NIR': 0.5225649920662337, 'year': 2024}}

        # Add geometry to features (because null geometry can't be exported)
        joined = aoi_mean_tmp.map(lambda ft: ee.Feature(aoi.centroid(1000)).copyProperties(ft))
        
        # Sort by glacier snow cover below snowline
        table_to_export = joined#.sort('gla_fsc_below_sl50', False)
        
        export_layer_name = 'decadal_meanNIR'  # Modify as needed
        
        # # Create year_month string for asset naming
        # year_month = f"{year}_{current_month-1:02d}"
        
        # Export to asset
        task = ee.batch.Export.table.toAsset(
            collection=ee.FeatureCollection(table_to_export).set('NAME', catchment_name).set('YEAR', year),
            description=f"{export_layer_name}_{catchment_name.replace('.', '')}_Year_{year}",
            assetId=f"projects/ee-hydro4u/assets/snow_CentralAsia/Folder4meanNIR/{export_layer_name}_{catchment_name.replace('.', '')}_Year_{year}_TEST2"
        )
        
        # Start the export task
        task.start()
        print(f"Export started")

catchment_names: ['ISSYKUL_15054', 'ISSYKUL_15002', 'ISSYKUL_15013', 'ISSYKUL_15016', 'ISSYKUL_15020', 'ISSYKUL_15022', 'ISSYKUL_15025', 'ISSYKUL_15030', 'ISSYKUL_15034', 'ISSYKUL_15039', 'ISSYKUL_15040', 'ISSYKUL_15044', 'ISSYKUL_15045', 'ISSYKUL_15049', 'ISSYKUL_15051', 'ISSYKUL_15057', 'ISSYKUL_15064', 'ISSYKUL_15069', 'ISSYKUL_15070', 'ISSYKUL_15081', 'ISSYKUL_15083', 'ISSYKUL_15090', 'ZERAFSHAN_17461', 'PYANDZH_17043', 'PYANDZH_17045', 'PYANDZH_17047', 'PYANDZH_17050', 'PYANDZH_17058', 'PYANDZH_17059', 'PYANDZH_17062', 'PYANDZH_17073', 'PYANDZH_17078', 'VAKSH_17082', 'VAKSH_17100', 'VAKSH_17101', 'VAKSH_17110', 'KOFARNIKHAN_17137', 'KOFARNIKHAN_17150', 'KOFARNIKHAN_17185', 'KOFARNIKHAN_17202', 'ZERAFSHAN_17288', 'ZERAFSHAN_17325', 'ZERAFSHAN_17329', 'ZERAFSHAN_17338', 'ZERAFSHAN_17344', 'PYANDZH_17391', 'PYANDZH_17453', 'VAKSH_17459', 'VAKSH_17462', 'ZERAFSHAN_16223', 'SURKHANDARYA_17194', 'SURKHANDARYA_17211', 'SURKHANDARYA_17223', 'QASHQADARYA_17231', 'QASHQADARYA_17236', 'QASHQ

#### Check on new assets in the cloud project and share publicly to make it available in the tool

In [10]:
from datetime import datetime, timedelta, timezone

# Once exports are completed, share all assets publically
assetdir = f"projects/ee-hydro4u/assets/{'snow_CentralAsia/Folder4SLA_v4'}"#
assets = ee.data.listAssets({"parent": assetdir})["assets"]

# Get the current UTC time and subtract 10 days
cutoff_time = datetime.now(timezone.utc) - timedelta(days=10)

# Filter assets updated in the last 10 days
recent_assets = [
    asset for asset in assets
    if 'updateTime' in asset and datetime.fromisoformat(asset['updateTime'].replace('Z', '+00:00')) > cutoff_time
]

print(f"Assets updated in the last 10 days: {len(recent_assets)}")
for table in recent_assets:
    id = table['id']
    asset = ee.data.getAsset(id)
    ee.data.setIamPolicy(id, {
        'bindings': [{'role': 'roles/owner', 'members': ['user:workshop.gee@gmail.com']},
                        {'role': 'roles/viewer', 'members': ['allUsers']}]})
    print('ID: ' + id)

#get the oldest asset: sort by updateTime
oldest_asset = sorted(assets, key=lambda x: x['updateTime'])[0]
print('Oldest asset ID:', oldest_asset['id'])




Assets updated in the last 10 days: 0
Oldest asset ID: projects/ee-hydro4u/assets/snow_CentralAsia/Folder4SLA_v4/decadal_SLA_AKHANGARAN_16230_until2025-01-01


### Gap filling of exported data

In [ ]:
#read the exported data: data/fsc_sla_timeseries.csv
# ---------------------------------------------
# Load the exported data from Google Earth Engine
# ---------------------------------------------
# Define the path to the exported CSV file
exported_csv_path = '../data/fsc_sla_timeseries.csv'
# Load the CSV file into a pandas DataFrame
df = pd.read_csv(exported_csv_path)

# Convert the 'Year-Month-Day' column to datetime format
df['date'] = pd.to_datetime(df['Year-Month-Day'])
# Sort the DataFrame by 'Name' and then by 'date'
df = df.sort_values(by=['Name', 'date'])
# Remove duplicates
df = df.drop_duplicates(subset=['Name', 'date'], keep='last')
# Calculate the time-steps between data points of the same catchment
df['time_step'] = df.groupby('Name')['date'].diff().dt.days
# Fill NaN values in the 'time_step' column with 0
df['time_step'] = df['time_step'].fillna(0)

# drop the 'system:index' column
df = df.drop(columns=['system:index'])
# drop the 'Year-Month-Day' column
df = df.drop(columns=['Year-Month-Day'])

# Define the decades in a month (1st, 2nd, and 3rd decade)
decades = [1, 2, 3]
results = []

def safe_interpolate(series):
    """
    Interpolates a series only if both previous and next values are available.
    """
    s = series.copy()
    for i in range(1, len(s) - 1):
        if pd.isna(s.iloc[i]) and not pd.isna(s.iloc[i - 1]) and not pd.isna(s.iloc[i + 1]):
            s.iloc[i] = (s.iloc[i - 1] + s.iloc[i + 1]) / 2
    return s


# Loop through each unique catchment name
for catchment_name in df['Name'].unique():
    catchment_data = df[df['Name'] == catchment_name].copy()  # ← copy is essential

    # Extract date parts safely
    catchment_data['year'] = catchment_data['date'].dt.year
    catchment_data['month'] = catchment_data['date'].dt.month

    # Define decades
    decades = [1, 2, 3]

    # Get year range
    years = range(catchment_data['year'].min(), 2024 + 1)
    months = range(1, 13)

    # Build full time grid
    combinations = pd.DataFrame([
        (year, month, decade) for year in years for month in months for decade in decades
    ], columns=['year', 'month', 'decade'])

    # Merge with actual data to find missing
    merged_df = pd.merge(combinations, catchment_data, on=['year', 'month', 'decade'], how='left')

    # Preserve catchment name
    merged_df['Name'] = catchment_name
    # Define a mapping for which day each decade starts on
    decade_start_days = {1: 1, 2: 11, 3: 21}

    # Apply to your merged_df to create a valid date
    merged_df['date'] = pd.to_datetime({
        'year': merged_df['year'],
        'month': merged_df['month'],
        'day': merged_df['decade'].map(decade_start_days)
    })

    # Fill NaN values in 'Basin' and 'Code' and '.geo' columns
    merged_df['Basin'] = merged_df['Basin'].fillna(method='ffill')
    merged_df['Code'] = merged_df['Code'].fillna(method='ffill')
    merged_df['.geo'] = merged_df['.geo'].fillna(method='ffill')
    # Fill NaN values in 'SLA_East', 'SLA_North', 'SLA_South', 'SLA_West', 'fsc', 'gla_area_below_sl50', and 'gla_fsc' columns
    # interpolate between previous and next values
    merged_df['SLA_East'] = merged_df['SLA_East'].interpolate(method='linear')
    merged_df['SLA_North'] = merged_df['SLA_North'].interpolate(method='linear')
    merged_df['SLA_South'] = merged_df['SLA_South'].interpolate(method='linear')
    merged_df['SLA_West'] = merged_df['SLA_West'].interpolate(method='linear')
    merged_df['fsc'] = merged_df['fsc'].interpolate(method='linear')
    merged_df['gla_area_below_sl50'] = merged_df['gla_area_below_sl50'].interpolate(method='linear')
    merged_df['gla_fsc'] = merged_df['gla_fsc'].interpolate(method='linear')

    # 'gla_fsc_below_sl50' is 1 whenever 'gla_area_below_sl50' is 0
    merged_df['gla_fsc_below_sl50'] = merged_df['gla_fsc_below_sl50'].mask(
        merged_df['gla_area_below_sl50'] == 0, 1
    )
    merged_df['gla_fsc_below_sl50'] = merged_df['gla_fsc_below_sl50'].interpolate(method='linear')

    # Identify rows where gla_fsc is NaN
    na_mask = merged_df['gla_fsc'].isna()

    # Set both columns to 0 where gla_fsc is NaN
    merged_df.loc[na_mask, 'gla_area_below_sl50'] = 0
    merged_df.loc[na_mask, 'gla_fsc_below_sl50'] = 0
    merged_df.loc[na_mask, 'gla_fsc'] = 0

    # for col in [
    #     'SLA_East', 'SLA_North', 'SLA_South', 'SLA_West',
    #     'fsc', 'gla_area_below_sl50', 'gla_fsc', 'gla_fsc_below_sl50'
    # ]:
    #     merged_df[col] = safe_interpolate(merged_df[col])

    results.append(merged_df)

# Concatenate all catchments into one full DataFrame
df_full = pd.concat(results, ignore_index=True)


pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', None)
# print(df_full)

df = df_full.copy()
# Reset the index
df = df.reset_index(drop=True)
# Print the DataFrame
print(df)
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')

# drop the 'time_step' column
df = df.drop(columns=['time_step'])

# Plotting the data
# ---------------------------------------------
# Create a new figure
plt.figure(figsize=(12, 6))
# Loop through each unique catchment name
for catchment_name in df['Name'].unique():
    # Filter the DataFrame for the current catchment
    catchment_data = df[df['Name'] == catchment_name]
    # Plot the data for the current catchment
    plt.plot(catchment_data['date'], catchment_data['SLA_East'], label=catchment_name)

# Set the title and labels
plt.title('Snowline Elevation by Aspect')
plt.xlabel('Date')
plt.ylabel('Snowline Elevation (m)')
# Rotate x-axis labels for better readability
plt.xticks(rotation=45)
# Add a legend
# plt.legend()
# Show the plot
plt.tight_layout()
plt.show()

# Print the maximum snowline elevation for each catchment
for catchment_name in df['Name'].unique():
    catchment_data = df[df['Name'] == catchment_name]
    max_snowline = catchment_data['SLA_North'].max()
    print(f"Maximum snowline elevation for {catchment_name}: {max_snowline} m")


# Save the DataFrame to a CSV file
output_csv_path = '../data/fsc_sla_timeseries_gapfilled.csv'
df.to_csv(output_csv_path, index=False)
print(f"Data saved to {output_csv_path}")
# ---------------------------------------------
# End of script
